In [2]:
import numpy as np
import pandas as pd
import random

In [1]:
str1 = '193-205'

In [8]:
str2 = '195.3 ( в валюте)'

In [11]:

list(map(lambda x: isinstance(x,(float,int)),str2.split(' ')))

[False, False, False, False]

In [7]:
list(range(*list(map(int,str1.split('-')))))

[193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204]

In [3]:
def projects_gen(n):
    res = []
    for i in range(n):
        t0 = random.randint(1,15)
        t1 = t0 + random.randint(1,15)
        res.append({'id':i,'t0':t0,'t1':t1,'d':t1-t0})
    return pd.DataFrame(res)

In [4]:
prjs = projects_gen(20)
prjs.head()

,id,t0,t1,d
0,0,11,21,10
1,1,9,21,12
2,2,5,18,13
3,3,2,6,4
4,4,4,7,3


In [77]:
def plan(projects_,maxx = 't0'):
    projects = projects_.sort_values(by=maxx)
    plan = []
    while len(projects)!=0:
        cur = projects.iloc[0]
        plan.append(cur)
        projects = projects.loc[projects[maxx]>cur[maxx]]
    return pd.DataFrame(plan).reset_index(drop=True)
        

In [79]:
plan(prjs,'d')

,id,t0,t1,d
0,6,14,15,1
1,14,2,5,3
2,1,13,17,4
3,17,5,10,5
4,2,7,13,6
5,0,14,21,7
6,11,1,10,9
7,3,9,20,11
8,9,8,21,13
9,19,9,23,14


In [5]:
from tqdm import tqdm

In [6]:
def imitation(m,n):
    res = []
    for _ in tqdm(range(m)):
        prj = projects_gen(n)
        res.append({'by_t0':len(plan(prj,'t0'))/n,'by_t1':len(plan(prj,'t1'))/n,'by_d':len(plan(prj,'d'))/n})
    return pd.DataFrame(res)
        

In [72]:
ex_res = imitation(100,100)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.71it/s]


In [76]:
ex_res.idxmin()

<bound method DataFrame.idxmin of     by_t0  by_t1  by_d
0    0.12   0.25  0.08
1    0.15   0.27  0.11
2    0.13   0.26  0.11
3    0.11   0.23  0.10
4    0.12   0.25  0.12
..    ...    ...   ...
95   0.13   0.25  0.11
96   0.10   0.25  0.15
97   0.10   0.24  0.11
98   0.11   0.24  0.12
99   0.10   0.25  0.11

[100 rows x 3 columns]>

In [73]:
ex_res.mean()

by_t0    0.1134
by_t1    0.2550
by_d     0.1163
dtype: float64

In [75]:
test = pd.read_excel(r'C:\Users\mag\Documents\Тест.xlsx')

In [18]:
import numpy as np

In [39]:
def plan(_projects,maxx = 't0',workers=1):
    projects = _projects.sort_values(by=maxx)

    if len(projects)<=workers:
        return projects
    cur = projects.iloc[:workers]
    _plan = cur['id']

    while len(projects)!=0:
        projects = projects.loc[projects[maxx]>min(cur[maxx])]
        cur = projects.iloc[:workers]
        _plan = np.concatenate([_plan,cur['id']])
    return pd.DataFrame(_plan).reset_index(drop=True)


In [43]:
plan(_projects=prjs,maxx='t1',workers=2)

,0
0,16
1,3
2,3
3,17
4,17
5,4
6,18
7,7
8,7
9,19


In [46]:
prjs.sort_values(by='t1')[:3]

,id,t0,t1,d
16,16,3,4,1
3,3,2,6,4
17,17,3,7,4


In [59]:
def plan_n_(projects_,maxx = 't0'):
    projects = projects_.sort_values(by=maxx)
    plan = []
    while len(projects)!=0:
        cur = projects.iloc[0]
        plan.append(cur)
        projects = projects.loc[projects[maxx]>cur[maxx]]
    res = pd.DataFrame(plan).reset_index(drop=True)
    return res,projects_.drop(res.id.tolist())

In [63]:
def plan_n(projects_,maxx = 't0',workers = 1):
    resid = projects_.sort_values(by=maxx)
    plan = None
    res = None
    for i in range(workers):
        plan,resid = plan_n_(resid,maxx)
        plan['worker'] = i
        res = pd.concat([plan,res])
        if len(resid.id)==0:
            break
    return res

In [68]:
plan_n(prjs,'t0',workers=6)

,id,t0,t1,d,worker
0,12,3,17,14,3
0,13,3,14,11,2
0,3,2,6,4,1
1,16,3,4,1,1
2,8,4,17,13,1
3,0,11,21,10,1
4,10,13,26,13,1
0,19,2,14,12,0
1,17,3,7,4,0
2,4,4,7,3,0


In [69]:
def imitation(m,n):
    res = []
    for _ in tqdm(range(m)):
        prj = projects_gen(n)
        res.append({'by_t0':len(plan_n(prj,'t0',workers=4))/n,'by_t1':len(plan_n(prj,'t1',workers=6))/n,'by_d':len(plan_n(prj,'d',workers=6))/n})
    return pd.DataFrame(res)

In [73]:
ex  =imitation(50,200)

100%|██████████| 50/50 [00:08<00:00,  5.66it/s]


In [74]:
ex.mean()

by_t0    0.3000
by_t1    0.6701
by_d     0.4495
dtype: float64